# Tutorial

In [5]:
from atuproot.AtUproot import AtUproot
process = AtUproot("output",
                  quiet = False,
                  parallel_mode = 'multiprocessing',
                  process = 0,
                  max_blocks_per_dataset = -1,
                  max_blocks_per_process = -1,
                  blocksize = 1000000,
                  profile = False,
                  profile_out_path = 'profile.txt')

In [7]:
class TestScribbler(object):
    def begin(self, event):
        pass
    
    def event(self, event):
        pass
    
from alphatwirl.loop import NullCollector

In [8]:
sequence = [(TestScribbler(), NullCollector())]

In [14]:
from collections import namedtuple
Dataset = namedtuple("Dataset", "name tree files")
hzz = Dataset(name = "hzz",
             tree = "events",
             files = ["HZZ.root"])
zuu = Dataset(name = "zuu",
             tree = "events",
             files = ["Zmumu.root"])
datasets = [hzz, zuu]

In [15]:
process.run(datasets, sequence)

AttributeError: 'EventBuilderConfig' object has no attribute 'blocksize'